# Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this lab we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime.

### Lab Architecture

In this lab we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Lab Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Strands Agents


## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [1]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models import BedrockModel
from retrying import retry

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.",
    callback_handler=None,  # default is PrintingCallbackHandler
)

# Retry if the agent throws a throttling exception.
# You could also return a response to the client and let it handle the retriable HTTP codes
# using https://docs.aws.amazon.com/bedrock-agentcore/latest/APIReference/API_InvokeAgentRuntime.html#API_InvokeAgentRuntime_ResponseSyntax
@retry(retry_on_exception=lambda e: ('throttl' in str(e)),
       wait_fixed=5000,
       stop_max_delay=30*1000)
def call_agent(user_input):
    return agent(user_input)

def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = call_agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))
    print(response)

Writing strands_claude.py


#### Invoking local agent

In [2]:
!python strands_claude.py '{"prompt": "What is the weather now?"}'

The current weather is sunny! It's a beautiful day outside.


## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

In [3]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
from retrying import retry

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.",
    callback_handler=None,  # default is PrintingCallbackHandler
)

# Retry if the agent throws a throttling exception.
# You could also return a response to the client and let it handle the retriable HTTP codes
# using https://docs.aws.amazon.com/bedrock-agentcore/latest/APIReference/API_InvokeAgentRuntime.html#API_InvokeAgentRuntime_ResponseSyntax
@retry(retry_on_exception=lambda e: ('throttl' in str(e)),
       wait_fixed=5000,
       stop_max_delay=30*1000)
def call_agent(user_input):
    return agent(user_input)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = call_agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Overwriting strands_claude.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this lab you will use the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [4]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import uuid
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_id = str(uuid.uuid4())[:8]
agent_name = f"strands_claude_getting_started_{agent_id}"
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements_for_runtime.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/workshop/strands_agentcore/strands_claude.py, bedrock_agentcore_name=strands_claude
Configuring BedrockAgentCore agent: strands_claude_getting_started_cb098c4e


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

Generated .dockerignore
Generated Dockerfile: /workshop/strands_agentcore/Dockerfile
Generated .dockerignore: /workshop/strands_agentcore/.dockerignore
Setting 'strands_claude_getting_started_cb098c4e' as default agent
Bedrock AgentCore configured: /workshop/strands_agentcore/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/workshop/strands_agentcore/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/workshop/strands_agentcore/Dockerfile'), dockerignore_path=PosixPath('/workshop/strands_agentcore/.dockerignore'), runtime='None', region='us-east-1', account_id='522040354751', execution_role=None, ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [5]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_getting_started_cb098c4e' to account 522040354751 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_getting_started_cb098c4e
✅ ECR repository available: 522040354751.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started_cb098c4e
Getting or creating execution role for agent: strands_claude_getting_started_cb098c4e
Using AWS region: us-east-1, account ID: 522040354751
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-c97549

Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_claude_getting_started_cb098c4e


Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-c97549402b
Starting execution role creation process for agent: strands_claude_getting_started_cb098c4e
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-east-1-c97549402b
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-c97549402b
✓ Role created: arn:aws:iam::522040354751:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-c97549402b
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-strands_claude_getting_started_cb098c4e
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::522040354751:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-c97549402b
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_claude_getting_started_cb098c4e
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-c97549402b
CodeBuild role doesn't exist, creating new role: AmazonBedro

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [6]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_getting_started_cb098c4e


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [7]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
print(invoke_response)

{'ResponseMetadata': {'RequestId': 'e5563a8c-ffe3-4b62-abf3-13fea0d0702d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 24 Sep 2025 12:13:11 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'e5563a8c-ffe3-4b62-abf3-13fea0d0702d', 'baggage': 'Self=1-68d3e049-4e02617164d42c3f29db1e3f,session.id=3dd2d2b7-1902-4529-857f-8922cf91f8c3', 'x-amzn-bedrock-agentcore-runtime-session-id': '3dd2d2b7-1902-4529-857f-8922cf91f8c3', 'x-amzn-trace-id': 'Root=1-68d3e049-6745163b2bbd7c9138e62ffc;Self=1-68d3e049-4e02617164d42c3f29db1e3f'}, 'RetryAttempts': 0}, 'runtimeSessionId': '3dd2d2b7-1902-4529-857f-8922cf91f8c3', 'traceId': 'Root=1-68d3e049-6745163b2bbd7c9138e62ffc;Self=1-68d3e049-4e02617164d42c3f29db1e3f', 'baggage': 'Self=1-68d3e049-4e02617164d42c3f29db1e3f,session.id=3dd2d2b7-1902-4529-857f-8922cf91f8c3', 'contentType': 'application/json', 'statusCode': 200, 'response': ["The weather is currently sunny! It's a beautif

### Processing invocation results

We can now process our invocation results to include it in an application

In [8]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

The weather is currently sunny! It's a beautiful day with clear skies and sunshine.

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [9]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

The answer is 4.

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    
    force=True
)